In [7]:
from app.loader import add_docs_to_knowledge_base
from app.db import reset_db, create_vectors_index, search_vector_db
from app.openai import get_embedding
import numpy as np

In [2]:
reset_db()
create_vectors_index()

Index idx:vectors created successfully


In [3]:
await add_docs_to_knowledge_base()

Loading documents


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:07<00:00,  7.14s/it]


Loaded 1 PDF documents

Splitting documents into chunks
Inception.pdf: 127 chunks

Total chunks: 127
Min chunk size: 351 tokens
Max chunk size: 512 tokens
Average chunk size: 496 tokens

Embedding chunks


100%|██████████| 127/127 [00:02<00:00, 49.55it/s]


Adding chunks to vector DB


In [ ]:
query = 'What is the most resilient parasite?'
vector = await get_embedding(query)
query_vector = np.array(vector, dtype=np.float32).tobytes()
search_vector_db(query_vector)

In [1]:
import json
from app.openai import chat, token_size
from app.tools import QueryKnowledgeBaseTool
from app.prompts import MAIN_SYSTEM_PROMPT, RAG_SYSTEM_PROMPT

In [2]:
chat_messages = []
system_message = {'role': 'system', 'content': MAIN_SYSTEM_PROMPT}
openai_tools = [QueryKnowledgeBaseTool.openai_tool_schema()]
user_message = {'role': 'user', 'content': "What's the most resilient parasite?"}
chat_messages.append(user_message)
content, tool_calls = await chat(
    messages=[system_message, *chat_messages],
    tools=openai_tools,
    tool_choice='auto'
)
print(content)
print(tool_calls)

None
[ChatCompletionMessageToolCall(id='call_Qb4rkxNAknxdTOfnpBUYnz19', function=Function(arguments='{"query_input":"What is the most resilient parasite?"}', name='QueryKnowledgeBaseTool'), type='function')]


In [5]:
tool_call = tool_calls[0]
kb_tool = QueryKnowledgeBaseTool(**json.loads(tool_call.function.arguments))
kb_result = await kb_tool()
chat_messages.extend([
    {'role': 'assistant', 'content': content, 'tool_calls': tool_calls},
    {'role': 'tool', 'tool_call_id': tool_call.id, 'content': kb_result}
])

In [6]:
rag_system_message = {'role': 'system', 'content': RAG_SYSTEM_PROMPT}
content, tool_calls = await chat(messages=[rag_system_message, *chat_messages])
print(content)

The most resilient parasite is an idea. Once an idea has taken hold in the brain, it becomes almost impossible to eradicate. As Cobb explains, "A person can cover it up, ignore it - but it stays there." This highlights the idea's contagious and persistent nature, making it a powerful form of "parasite" in a metaphorical sense.
